In [4]:
!git clone https://jadelee@github.com/hanb0i/Basic-PiNN-Model-For-Three-Layered-Collision.git


Cloning into 'Basic-PiNN-Model-For-Three-Layered-Collision'...
remote: Enumerating objects: 261, done.
remote: Counting objects: 100% (261/261), done.
remote: Compressing objects: 100% (200/200), done.
remote: Total 261 (delta 85), reused 213 (delta 40), pack-reused 0 (from 0)
Receiving objects: 100% (261/261), 7.35 MiB | 33.77 MiB/s, done.
Resolving deltas: 100% (85/85), done.


In [5]:
import os, sys
from pathlib import Path

repo_root = Path('/content/Basic-PiNN-Model-For-Three-Layered-Collision')
pinn_dir = repo_root / 'pinn-workflow'
sys.path.insert(0, str(pinn_dir))
os.chdir(pinn_dir)


In [ ]:

import torch
import torch.optim as optim
import numpy as np
import time
import os

import pinn_config as config
import data
import model
import physics
import matplotlib.pyplot as plt

def train():
    if torch.cuda.is_available():
        device = torch.device('cuda')
    elif torch.backends.mps.is_available():
        device = torch.device('mps')
    else:
        device = torch.device('cpu')
    print(f"Using device: {device}")
    
    # Initialize Model
    pinn = model.MultiLayerPINN().to(device)
    
    # Initialize Optimizers
    optimizer_adam = optim.Adam(pinn.parameters(), lr=config.LEARNING_RATE)
    
    # Data Container
    training_data = data.get_data()
    
    # History
    loss_history = []
    
    print("Starting Adam Training...")
    start_time = time.time()
    last_time = start_time
    
    for epoch in range(config.EPOCHS_ADAM):
        optimizer_adam.zero_grad()
        
        # Periodic data refresh (optional, computationally expensive to re-sample every Step)
        if epoch % 1000 == 0 and epoch > 0:
            training_data = data.get_data()
            
        loss_val, losses = physics.compute_loss(pinn, training_data, device)
        loss_val.backward()
        optimizer_adam.step()
        
        loss_history.append(loss_val.item())
        
        if epoch % 100 == 0:
            current_time = time.time()
            step_duration = current_time - last_time
            last_time = current_time
            print(f"Epoch {epoch}: Total Loss: {loss_val.item():.6f} | "
                  f"PDE: {losses['pde']:.6f} | BC: {losses['bc_sides']:.6f} | "
                  f"Load: {losses['load']:.6f} | Interface: {losses['interface']:.6f} | "
                  f"Time: {step_duration:.4f}s")
            
    print(f"Adam Training Complete. Total Time: {time.time() - start_time:.2f}s")
    
    # L-BFGS Training
    print("Starting L-BFGS Training...")
    optimizer_lbfgs = optim.LBFGS(pinn.parameters(), 
                                  max_iter=100, 
                                  history_size=50, 
                                  line_search_fn="strong_wolfe")
    
    def closure():
        optimizer_lbfgs.zero_grad()
        loss_val, _ = physics.compute_loss(pinn, training_data, device)
        loss_val.backward()
        return loss_val
        
    num_lbfgs_steps = config.EPOCHS_LBFGS // 20
    print(f"Running {num_lbfgs_steps} L-BFGS outer steps.")
    
    for i in range(num_lbfgs_steps): 
        step_start = time.time()
        loss_val = optimizer_lbfgs.step(closure)
        step_end = time.time()
        loss_history.append(loss_val.item())
        
        # Print every step to see progress since total steps is small (5)
        print(f"L-BFGS Step {i}: Loss: {loss_val.item():.6f} | Time: {step_end - step_start:.4f}s")
            
    # Save Model
    torch.save(pinn.state_dict(), "pinn_model.pth")
    np.save("loss_history.npy", np.array(loss_history))
    print("Model saved.")
    return pinn

if __name__ == "__main__":
    train()


Using device: cpu
Starting Adam Training...
Epoch 0: Total Loss: 89.438919 | PDE: 7.228379 | BC: 0.000000 | Load: 0.012610 | Interface: 0.044440 | Time: 1.8827s
Epoch 100: Total Loss: 0.043785 | PDE: 0.002819 | BC: 0.000000 | Load: 0.000007 | Interface: 0.000074 | Time: 134.0211s
Epoch 200: Total Loss: 0.023327 | PDE: 0.001428 | BC: 0.000000 | Load: 0.000006 | Interface: 0.000021 | Time: 130.3218s
Epoch 300: Total Loss: 0.017342 | PDE: 0.001006 | BC: 0.000000 | Load: 0.000006 | Interface: 0.000007 | Time: 133.2900s
Epoch 400: Total Loss: 0.015363 | PDE: 0.000841 | BC: 0.000000 | Load: 0.000005 | Interface: 0.000004 | Time: 146.1875s
Adam Training Complete. Total Time: 687.65s
Starting L-BFGS Training...
Running 50 L-BFGS outer steps.
L-BFGS Step 0: Loss: 0.014456 | Time: 168.1190s
L-BFGS Step 1: Loss: 0.012228 | Time: 148.7071s


In [7]:
!pwd
!ls


/content/Basic-PiNN-Model-For-Three-Layered-Collision/pinn-workflow
data.py		  model.py    pinn_config.py  __pycache__  train_colab.ipynb
fea_solution.npy  physics.py  plot.py	      README.md    train.py


In [8]:
%cd /content/Basic-PiNN-Model-For-Three-Layered-Collision
!ls


/content/Basic-PiNN-Model-For-Three-Layered-Collision
benchmark_fea.py      fea_solution.npy	pinn-workflow
compare_results.py    fea-workflow	__pycache__
cross_section.png     loss_curve.png	pyproject.toml
diagnostic_report.md  loss_history.npy	traction_top_patch.png
diagnostics.py	      pinn_model.pth


In [9]:
%cd /content/Basic-PiNN-Model-For-Three-Layered-Collision/pinn-workflow
!python plot.py


/content/Basic-PiNN-Model-For-Three-Layered-Collision/pinn-workflow
Model not found, cannot plot.


In [6]:
from IPython.display import Image, display

display(Image("loss_curve.png"))
display(Image("displacement_top.png"))
display(Image("cross_section.png"))


FileNotFoundError: No such file or directory: 'loss_curve.png'

FileNotFoundError: No such file or directory: 'loss_curve.png'

<IPython.core.display.Image object>

FileNotFoundError: No such file or directory: 'displacement_top.png'

FileNotFoundError: No such file or directory: 'displacement_top.png'

<IPython.core.display.Image object>

FileNotFoundError: No such file or directory: 'cross_section.png'

FileNotFoundError: No such file or directory: 'cross_section.png'

<IPython.core.display.Image object>

In [13]:
%cd

/root


In [14]:
%cd /content/Basic-PiNN-Model-For-Three-Layered-Collision

/content/Basic-PiNN-Model-For-Three-Layered-Collision


In [19]:
!python compare_results.py

Traceback (most recent call last):
  File "/content/Basic-PiNN-Model-For-Three-Layered-Collision/compare_results.py", line 8, in <module>
    import config
ModuleNotFoundError: No module named 'config'


In [16]:
compare_dir = repo_root 
sys.path.insert(0, str(compare_dir))
os.chdir(compare_dir)


In [23]:
%cd /content/Basic-PiNN-Model-For-Three-Layered-Collision

/content/Basic-PiNN-Model-For-Three-Layered-Collision


In [26]:
!python compare_results.py

Traceback (most recent call last):
  File "/content/Basic-PiNN-Model-For-Three-Layered-Collision/compare_results.py", line 8, in <module>
    import config
ModuleNotFoundError: No module named 'config'


In [1]:

import numpy as np
import matplotlib.pyplot as plt
import torch
import sys

import pinn_config as config
import model
from scipy.interpolate import RegularGridInterpolator

def compare():
    print("Loading FEA Solution...")
    data = np.load("fea_solution.npy", allow_pickle=True).item()
    X_fea = data['x'] # (nx, ny, nz)
    Y_fea = data['y']
    Z_fea = data['z']
    U_fea = data['u'] # (nx, ny, nz, 3)
    
    # Grid axes
    x_axis = X_fea[:, 0, 0]
    y_axis = Y_fea[0, :, 0]
    z_axis = Z_fea[0, 0, :]
    
    print(f"FEA Grid: {len(x_axis)}x{len(y_axis)}x{len(z_axis)}")
    
    # 1. Generate PINN Predictions on same Grid
    if torch.cuda.is_available():
        device = torch.device('cuda')
    elif torch.backends.mps.is_available():
        device = torch.device('mps')
    else:
        device = torch.device('cpu')
    pinn = model.MultiLayerPINN().to(device)
    model_path = "pinn_model.pth"
    if not os.path.exists(model_path):
        # Check in pinn-workflow directory
        potential_path = os.path.join(os.path.dirname(__file__), 'pinn-workflow', 'pinn_model.pth')
        if os.path.exists(potential_path):
            model_path = potential_path

    try:
        pinn.load_state_dict(torch.load(model_path, map_location=device, weights_only=True))
        print(f"Loaded PINN model from {model_path}")
    except Exception as e:
        print(f"PINN model not found or error loading: {e}")
        return
        
    pinn.eval()
    
    pts = np.stack([X_fea.ravel(), Y_fea.ravel(), Z_fea.ravel()], axis=1) # (N, 3)
    
    # We need to query layer-wise because PINN takes layer_idx
    # Layer interfaces: 0, 0.033, 0.066, 0.1
    # Z coordinate determines layer
    z_flat = pts[:, 2]
    
    # Masks
    eps = 1e-5
    mask1 = (z_flat >= config.Layer_Interfaces[0] - eps) & (z_flat <= config.Layer_Interfaces[1] + eps)
    mask2 = (z_flat >= config.Layer_Interfaces[1] - eps) & (z_flat <= config.Layer_Interfaces[2] + eps)
    mask3 = (z_flat >= config.Layer_Interfaces[2] - eps) & (z_flat <= config.Layer_Interfaces[3] + eps)
    
    # Prioritize higher layers for overlaps (standard practice or arbitrary)
    # Actually, interfaces match, so just pick one.
    mask2 = mask2 & (~mask1) # simple exclusivity cleanup if needed, but overlap is fine if continuous
    # Better: strict intervals
    m1 = z_flat <= config.Layer_Interfaces[1]
    m2 = (z_flat > config.Layer_Interfaces[1]) & (z_flat <= config.Layer_Interfaces[2])
    m3 = z_flat > config.Layer_Interfaces[2]
    
    U_pinn_flat = np.zeros_like(pts)
    
    with torch.no_grad():
        # Layer 1
        p1 = torch.tensor(pts[m1], dtype=torch.float32).to(device)
        if len(p1) > 0:
            U_pinn_flat[m1] = pinn(p1, 0).cpu().numpy()
            
        # Layer 2
        p2 = torch.tensor(pts[m2], dtype=torch.float32).to(device)
        if len(p2) > 0:
            U_pinn_flat[m2] = pinn(p2, 1).cpu().numpy()
            
        # Layer 3
        p3 = torch.tensor(pts[m3], dtype=torch.float32).to(device)
        if len(p3) > 0:
            U_pinn_flat[m3] = pinn(p3, 2).cpu().numpy()
            
    U_pinn = U_pinn_flat.reshape(U_fea.shape)
    
    # 2. Compute Metrics
    # U_z at top surface
    u_z_fea_top = U_fea[:, :, -1, 2]
    u_z_pinn_top = U_pinn[:, :, -1, 2]
    
    abs_diff = np.abs(u_z_fea_top - u_z_pinn_top)
    mae = np.mean(abs_diff)
    max_err = np.max(abs_diff)
    
    print(f"Comparison Results (Top Surface u_z):")
    print(f"MAE: {mae:.6f}")
    print(f"Max Error: {max_err:.6f}")
    print(f"Peak Deflection FEA: {u_z_fea_top.min():.6f}")
    print(f"Peak Deflection PINN: {u_z_pinn_top.min():.6f}")
    
    # 3. Plots
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Contours
    # FEA
    c1 = axes[0].contourf(X_fea[:,:,0], Y_fea[:,:,0], u_z_fea_top, levels=50, cmap='jet')
    axes[0].set_title("FEA Displacement u_z (Top)")
    plt.colorbar(c1, ax=axes[0])
    
    # PINN
    c2 = axes[1].contourf(X_fea[:,:,0], Y_fea[:,:,0], u_z_pinn_top, levels=50, cmap='jet')
    axes[1].set_title("PINN Displacement u_z (Top)")
    plt.colorbar(c2, ax=axes[1])
    
    # Error
    c3 = axes[2].contourf(X_fea[:,:,0], Y_fea[:,:,0], abs_diff, levels=50, cmap='magma')
    axes[2].set_title("Absolute Error |FEA - PINN|")
    plt.colorbar(c3, ax=axes[2])
    
    plt.savefig("comparison_top.png")
    print("Saved comparison_top.png")
    
    # Cross section
    # y index middle
    mid_y = U_fea.shape[1] // 2
    
    xz_X = X_fea[:, mid_y, :]
    xz_Z = Z_fea[:, mid_y, :]
    xz_Uz_fea = U_fea[:, mid_y, :, 2]
    xz_Uz_pinn = U_pinn[:, mid_y, :, 2]
    
    fig2, axes2 = plt.subplots(1, 2, figsize=(15, 6))
    
    c4 = axes2[0].contourf(xz_X, xz_Z, xz_Uz_fea, levels=50, cmap='jet')
    axes2[0].set_title("FEA Cross Section u_z")
    plt.colorbar(c4, ax=axes2[0])
    
    c5 = axes2[1].contourf(xz_X, xz_Z, xz_Uz_pinn, levels=50, cmap='jet')
    axes2[1].set_title("PINN Cross Section u_z")
    plt.colorbar(c5, ax=axes2[1])
    
    plt.savefig("comparison_cross_section.png")
    print("Saved comparison_cross_section.png")

if __name__ == "__main__":
    compare()


ModuleNotFoundError: No module named 'pinn_config'